In [ ]:
#!pip install sentinelhub

In [ ]:
#!pip install folium
#!pip install area
#!pip install deafrica_tools

# Commercial data

For use of commercial data you have to first provide your sentinel-hub account client id and client secret.

In [ ]:
from sentinelhub import SHConfig
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from odc_sh import SentinelHubCommercialData
from odc_sh import Providers, AirbusConstellation, ThumbnailType, WorldViewKernel, WorldViewSensor, SkySatType, SkySatBundle, ScopeType, ScopeBundle
import pandas as pd
import numpy as np
import folium
from shapely.geometry import box
from area import area
import os
import json
import shapely

from deafrica_tools.plotting import display_map, map_shapefile, _degree_to_zoom_level

sh_client_id=""
sh_client_secret=""

if not sh_client_id:
    sh_client_id = os.environ['SH_CLIENT_ID']

if not sh_client_secret:
    sh_client_secret = os.environ['SH_CLIENT_SECRET']



config = SHConfig()
config.sh_client_id = sh_client_id
config.sh_client_secret = sh_client_secret


shcd = SentinelHubCommercialData(config)

## INFO

Every response that have method "print_info()", also have a raw data in response object.
Example:

In [ ]:
q = shcd.quotas()
q.print_info()

# response data
print(q.data)

## Getting quotas

In [ ]:
# Optional 1: To get single quota add id as parameter
#
#     shcd.quotas("asd124-12ddas...")
#

q = shcd.quotas()
q.print_info()


# Simple search

Setting date and time

In [ ]:
time_from = "2017-05-25T00:00:00Z"
time_to = "2018-10-25T23:59:59Z"

Setting search area with 2 options:
1. Bounding Box
2. Polygon

### Bounding Box

In [ ]:
# Option 1: Set bounds as bbox
bounds = [
  12.742243,
  42.05043,
  12.746302,
  42.053218
]

Display bounding box on the map

In [ ]:
x = (bounds[0],bounds[2])
y = (bounds[1],bounds[3])

#view the location
display_map(x=x, y=y)

### Polygon

In [ ]:

# Option 2: Set bounds as Polygon
bounds_polygon = {
  "type": "Polygon",
  "coordinates": [
   [
    [
     12.500395,
     41.931337
    ],
    [
     12.507856,
     41.931018
    ],
    [
     12.507513,
     41.927825
    ],
    [
     12.50048,
     41.928719
    ],
    [
     12.500395,
     41.931337
    ]
   ]
  ]
 }

Extract coordinates from JSON

In [ ]:
geom = bounds_polygon["coordinates"][0]

Calculate polygon centroid to center the map

In [ ]:
lat_all = []
for i in range(len(geom)):
    lat_all.append(geom[i][1])
    
lon_all = []
for i in range(len(geom)):
    lon_all.append(geom[i][0])
    
center = [np.mean(lat_all), np.mean(lon_all)]
center

Calculate zoom level based on coordinates

In [ ]:
lat_zoom_level = _degree_to_zoom_level(min(lat_all), max(lat_all))
lon_zoom_level = _degree_to_zoom_level(min(lon_all), max(lon_all))

zoom_level = min(lat_zoom_level, lon_zoom_level)

Display polygon on the map

In [ ]:
#Multiple options for basemap
#Option 1. World imagery from ESRI
World_Imagery = ("http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer" + '/MapServer/tile/{z}/{y}/{x}')

#Option 2: World Topo Map from ESRI
#World_Topo_Map = (
#    'http://services.arcgisonline.com/arcgis/rest/services/World_Topo_Map'
#    + '/MapServer/tile/{z}/{y}/{x}'
#)

#The attribute paramater needs to be provided for custom tiles
map = folium.Map(location=center, tiles=World_Imagery, attr='ESRI World Imagery', zoom_start=zoom_level)
folium.GeoJson(bounds_polygon, name="geojson").add_to(map)

map

### Calculate Area

In [ ]:
aoi = box(bounds[0],bounds[1],bounds[2],bounds[3])

#bbox area calculation
area_sqm = area(json.dumps(shapely.geometry.mapping(aoi))) # area in m2 (sqm)

print('Area for selected bounding box is:', round(area_sqm * 10 ** (-6),2), 'sq. km')

## Choose one of the providers below

### 1. AIRBUS Pleiades & SPOT

In [ ]:

# Option 1: AIRBUS Pleiades & SPOT
# Optional parameters: 
#    - maxCloudCoverage (Values: 0-100 | 100 as default)
#    - maxSnowCoverage (Vaules: 0-90 | 90 as default)
#    - maxIncidenceAngle (Values: 0-90 | 90 as default)
#    - processingLevel (Values: "Sensor","Album" | "Sensor","Album" as default)
#
#  example: shcd.search_airbus(AirbusConstellation, Bounds, Time_From, Time_To, *Optional parameters*)

res = shcd.search_airbus(AirbusConstellation.SPOT, bounds, time_from, time_to, maxCloudCoverage=90, maxSnowCoverage=50)

# Optional parameters: 
#    - props
#
# example: res.print_info(props=["id", "acquisitionDate", "resolution", "cloudCover"])
res.print_info(aoi=aoi)

# Getting ids
item_ids = res.get_ids()



### 2. Planet SCOPE

In [ ]:

# Option 2: Planet SCOPE
# Optional parameters: 
#    - maxCloudCoverage (Values: 0-100 | 100 as default)
#
#  example: shcd.search_airbus(ScopeType, ScopeBundle, Bounds, Time_From, Time_To, *Optional parameters*)

res = shcd.search_planet(ScopeType.PSScene, ScopeBundle.ANALYTIC_UDM2, bounds, time_from, time_to, maxCloudCoverage=90)

# Optional parameters: 
#    - props
#
# example: res.print_info(props=["cloud_cover", "snow_ice_percent", "acquired", "pixel_resolution"])
res.print_info(aoi=aoi)

# Getting ids
item_ids = res.get_ids()


### 3. Planet SkySat

In [ ]:

# Option 3: Planet SkySat
#
#  example: shcd.search_airbus(ScopeType, ScopeBundle, Bounds, Time_From, Time_To, planetApiKey=<your_planey_api_key>)

res = shcd.search_planet(SkySatType.SkySatCollect, SkySatBundle.PANCHROMATIC, bounds, time_from, time_to, planetApiKey="")

# Optional parameters: 
#    - props
#
# example: res.print_info(props=["cloud_cover", "snow_ice_percent", "acquired", "pixel_resolution"])
res.print_info(aoi=aoi)

# Getting ids
item_ids = res.get_ids()


### 4. WorldView

In [ ]:

# Option 4: WorldView MAXAR
# Optional parameters:
#    - maxCloudCoverage (Values: 0-100 | 100 as default)
#    - minOffNadir (Values: 0-45 | 0 as default)
#    - maxOffNadir (Values: 0-45 | 45 as default)
#    - minSunElevation (Values: 0-90 | o as default)
#    - maxSunElevation (Values: 0-90 | 90 as default)
#    - sensor (Values: WorldViewSensor | Any as default)
#
#  example: shcd.search_airbus(WorldViewKernel, ScopeBundle, Bounds, Time_From, Time_To, *Optional parameters*)
#

res = shcd.search_worldview(WorldViewKernel.MTF, bounds, time_from, time_to, sensor=WorldViewSensor.WV01.value)

# Optional parameters: 
#    - props
#
# example: res.print_info(props=["catalogId", "sensor", "maxSunAzimuth", "acquisitionDateStart"])
res.print_info(aoi=aoi)

# Getting ids
item_ids = res.get_ids()


## Error checking

In [ ]:
if not len(res.data.features):
    print("No data found in request above.")

## Thumbnail

In [ ]:
import io
from PIL import Image
import matplotlib.pyplot as plt

# Define product ids for a preview
# item_id = [item_ids[0], item_ids[1], item_ids[2]...]
selected_ids = [item_ids[0], item_ids[1]] if len(res.data.features) else []
if not len(selected_ids):
    print("No ids found.")

for sid in selected_ids:
    thumbnail = shcd.thumbnail(res.thumb, sid)
    image_bytes = io.BytesIO(thumbnail.content)
    image = Image.open(image_bytes)
    plt.imshow(image)
    plt.show()
    

# Data Order

### OPTIONAL: Getting compatible collections

In [ ]:
collections = shcd.get_collection(res.query)
collections.print_info()

#### Select collection id

In [ ]:
colIdx = -1 # Idx number from search above
collectionId = collections.data[colIdx]["id"]

### Query order: Airbus, Planet SkySat and Scope

In [ ]:
# Optional parametrs:
#  - collectionId
#
# example: shcd.order("Order name", Query, *Optional_parameters*)

# response = shcd.order("New query order", res.query)

### Normal order: Airbus, Planet SkySat and Scope

In [ ]:
# Optional parametrs:
#  - collectionId
#
# example: shcd.order("Order name", Query, Items_Ids)
# example with collectionId: shcd.order("Order name", Query, Items_Ids, collectionId=collectionId)


order = shcd.order("New normal order 11", res.query, item_ids=selected_ids)
order.print_info()

### Getting orders

In [ ]:
# Optional: Get single order by adding order_id as parameter
# example: shcd.get_orders("asdf12-12bfa...")

orders = shcd.get_orders()
orders.print_info()

order_ids = orders.get_ids()


## Selecting an order

In [ ]:
order_id = order_ids[2] #you can use order_ids[1]

### Delete order

In [ ]:
res = shcd.delete_order(order_id)
res.print_info()

### Confirm order

In [ ]:
res = shcd.confirm_order(order_id)
res.print_info()